# SHP2CSV algorithm
## Importing libraries

In [1]:
from shapely.geometry import shape
from shapely.ops import unary_union
import fiona
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from IPython.core.debugger import Tracer
from shutil import copyfile
import pandas as pd

## Importing and reading shp file

## Converting `geoms` into a planar graph

In [3]:
file_info = fiona.open("shp/Bogota_sencillo/New_Shapefile.shp")

## Creating a Graph object in order to print it
this part of the code is not needed, it is just for having an idea of the graph that is going to be exported

## Plotting the graph

## Creating sets

In [4]:
y = set()
x = set()
lengths = set()

nodes = set()
edges = set()
edges_prime = set()

nodes_norm = list()
edges_norm = set()

## Creating a list of nodes

In [5]:
for line in file_info:
    for node in line['geometry']['coordinates']:
        nodes.add((round(node[0],3),round(node[1],3)))
        x.add(round(node[0],3))
        y.add(round(node[1],3))

nodes = list(nodes)

## Normalizing the nodes coordinates

In [6]:
# Getting the max and min for x and y axis
x_max = max(x)
x_min = min(x)
y_max = max(y)
y_min = min(y)

In [7]:
df3 = pd.DataFrame(data=[[x_min,x_max,y_min,y_max]], columns=['x_min','x_max','y_min','y_max'])
df3.to_csv('../../distributions/boundaries.csv', index=False)

In [8]:
# Normalizing

for node in nodes:
    node_norm = (nodes.index(node), [(node[0]-x_min)/(x_max-x_min),(node[1]-y_min)/(y_max-y_min)])
    nodes_norm.append(node_norm)
    
nodes_norm = [(item[0], [round(item[1][0],3), round(item[1][1],3)]) for item in nodes_norm]

## Creating a list of edges

In [9]:
for line in file_info:
    last_coord = None
    for coord in line['geometry']['coordinates']:
        if last_coord != None:
            length = ((coord[0]-last_coord[0])**2+(coord[1]-last_coord[1])**2)**0.5
            edge = ((nodes.index((round(last_coord[0],3),round(last_coord[1],3))),nodes.index((round(coord[0],3),round(coord[1],3)))),length,line['properties']['Id'])
            edges.add(edge)
        last_coord = coord
        
    
edges = [edge for edge in edges if edge[0][1] != edge[0][0]]


## Normalizing the cost of the edges

In [10]:
length_max = max(list(zip(*edges))[1])
length_min = min(list(zip(*edges))[1])

# normalizing length
edges_norm = [(edges.index(edge),edge[0],(edge[1]-length_min)/(length_max-length_min),edge[2]) for edge in edges]

## Exporting results to csv

In [11]:
# Writing csv
import csv

# vertices
with open("vertices.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in nodes_norm:
        writer.writerow([val[0],val[1][0],val[1][1]])
        
# edges
with open("edges.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in edges_norm:
        writer.writerow([val[0], val[1][0], val[1][1], val[2], val[3]])

copyfile('edges.csv', '../../streets/edges.csv');
copyfile('vertices.csv', '../../streets/vertices.csv');

In [27]:
coords = []

for coord in list(zip(*edges))[0]:
    coord1 = (nodes[coord[0]], nodes[coord[1]])
    coords.append(coord1)

In [31]:
coords

[((600570.841, 10514335.844), (601042.94, 10514969.907)),
 ((604481.336, 10517647.98), (604741.068, 10517556.703)),
 ((604939.3, 10513313.715), (604532.388, 10513096.414)),
 ((605180.494, 10521555.059), (605627.173, 10521413.259)),
 ((602994.616, 10509701.625), (599413.747, 10504463.937)),
 ((605381.078, 10522676.013), (605856.672, 10522715.384)),
 ((598321.544, 10515929.426), (600218.173, 10518231.739)),
 ((605906.741, 10519212.831), (606374.491, 10520109.82)),
 ((600218.173, 10518231.739), (601511.309, 10516774.051)),
 ((603435.997, 10516453.671), (604136.795, 10515722.67)),
 ((598927.387, 10503053.02), (599946.365, 10502651.603)),
 ((607328.774, 10522294.544), (608292.63, 10522036.656)),
 ((600440.573, 10509948.801), (601751.975, 10508853.314)),
 ((602141.78, 10514268.512), (603732.523, 10513569.371)),
 ((597663.557, 10520294.865), (598898.18, 10519215.758)),
 ((606808.006, 10517274.967), (605478.308, 10518006.134)),
 ((600144.004, 10512572.664), (600621.793, 10511292.558)),
 ((6060